# Кластеризация новостных текстов #

** Команда: **
* Анна Лапидус
* Анастасия Кузнецова
* Надежда Катричева
* Альфия Бабий

## Постановка задачи и формулировка задания ##

Итак в нашем распоряжении есть архив из новостей, которые соотнесены с одним из 28 резонансных событий. Эти события обозначены определенным номером (соответствует номеру кластера). В наши *задачи* входит:
1. Провести кластеризацию текстов новостей и попытаться соотнести полученные в ходе анализа кластеры с данными кластерами;
2. Провести кластеризацию на меньшее количество кластеров и проверить, получается ли выделить общие направления новостей.

## Задание ##
1. Провести предобработку текста:
    * Токенизация
    * Приведение к нижнему регистру
    * Лемматизация 
    * Найти дубликаты (если есть)
    * Определить, сколько новостей относятся к каждому событию. 
2. С помощью алгоритма **K-means** разбить тексты на 28 кластеров и определить эффективность разбиения с помощью мер качества (Homogeneity, Completeness, V-measure, Adjusted Rand-Index).
3. Определить, как разные способы обработки данных влияют на качество кластеризации. Мы будем использовать:
    * TF-IDF преобразование
    * Сингулярное разложение
    * Нормировку признакового пространства. 
4. С помощью K-means повторно разбить новости на 5 кластеров и попытаться соотнести их с резонансными событиями.

## Теоречиеское обоснование ##

**K-means**

В этом небольшом исследовании мы решили воспользоваться алгоритмомо к-средних, метрикой которого является евклидово расстояние. Его цель -- минимизировать расстояние между документами и кластерами (их центроидами). На первом этапе алгоритма выбираются случайным образом центры кластеров, а затем на кажом последующем шаге алгоритм стремится минимизировать расстояние между документами, изменяя координаты центроидов. При этом тексты представлены в виде векторов, и основной величиной, с помощью которой измеряется расстояние между центроидами является *остаточная сумма квадратов*. 

Мы решили воспользоваться этим алгоритмом, поскольку он оказывается довольно простым в реализации, но в то же время он довольно эффективен, в отличие, например, от алгоритма ближайших соседей, который показывает меньшую точность.

Для увеличения эффективности алгоритма мы будем использовать следующие методы:

**Сингулярное разложение**

Сингулярное разложение необходимо для того, чтобы сократить размерность пространства признаков (слов), которыми мы характеризуем каждый документ.

В резульатте разложения получается три матрицы, перемноженных между собой, в матрице по середине элементы только на диагонали (сингулярные числа) и их столько, сколько слов в нашем словаре. Мы оставляем заданное число этих сингулярных чисел (n компонент, которые задаются в методе), остальные обнуляем в результате при обратном перемножении размерность пространства признаков (количество слов) сократится дод заданного *n*. Это может улучшить качество кластеризации, поскольку мы избавляемся от ненужных признаков и оставляем только значимые.

**Нормализация**

Нормализация необходима для того, чтобы привести вектора к общей размерности. Не понятно, может ли это в нашем случае привести к значительному улучшению качества кластеризации, это мы орпеделим в ходе анализа.

**TF-IDF**

TF-IDF мера помогает нормировать текст, определяя наиболее важные слова (с большим весом) и отсеивая слишком частотные или очень редкие слова. Эта мера также может помочь нам существенно улучшить качество кластеризации.



# Шаг 1. Предобработка текстов #

## Импортируем библиотеки ##

In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
import sklearn
from sklearn.pipeline import Pipeline

In [2]:
stopwords_rus = stopwords.words('russian')

## Читаем csv из файла, создаем дата-фрейм с помощью pandas ##

Всего в нашем дата-фрейме оказалось 1930 текстов. Посмотрим, сколько удастся выявить дубликатов. 

In [3]:
df = pd.read_csv('raw_news.csv', index_col = 0)
df.shape

(1930, 2)

## Создаем список уникальных новостей (удаление дубликатов) ##

Получаем 1924 уникальных новости и 6 дубликатов.

In [4]:
df_unique = df.drop_duplicates(subset = 'text', keep = 'first')
df_unique.shape

(1924, 2)

Для того, чтобы было удобнее работать с данными, переведем их в обычный список. 

In [5]:
news = df_unique['text'].values 

In [6]:
print(news[3])

 Google Новости ТОП, Москва, 14 января 2017 АКЦИЯ ПРОТИВ ПЕРЕДАЧИ ИСААКИЕВСКОГО СОБОРА РПЦ ПРОШЛА БЕЗ НАРУШЕНИЙ Москва. 13 января. INTERFAX.RU - Акция противников передачи Исаакиевского собора Русской православной церкви прошла без нарушений, сообщили "Интерфаксу" в пятницу в пресс-службе управления МВД по Санкт-Петербургу и Ленинградской области. "Общественная акция рядом... http://www.interfax.ru/russia/545283#googletop


## Проверим сколько текстов есть для каждого резонансного события ##

In [7]:
df_unique.groupby('event_id').size()

event_id
1     100
2      51
3      84
4      62
5       2
6      41
7     100
8      27
9      76
10    100
11     49
12    100
13     24
14      2
15      7
16    100
17    102
18    100
19     45
20      8
21    100
22     82
23    100
24     62
25    100
26    100
27    100
28    100
dtype: int64

## Токенизация текстов ##

Проводим токенизацию в несколько этапов. Для этого напишем функцию, которая приводит тексты к нижнему регистру, с помощью регулярного выражения удалим все гиперссылки, избавимся от ненужных знаков препинания. Мы знаем, что метод *findall* возвращает нам список, поэтому на выходе получим список токенов для одного текста.

In [8]:
def clean_news(single_new):
    lower_new = single_new.lower().replace(' - ', ' ').replace('. ', ' ')
    clean_new = re.sub(r'http\S+', '', lower_new)
    word_list = re.findall(r'[а-яa-z\-\.]+', clean_new)
    return word_list

In [9]:
print(clean_news(news[30]))

['риа', 'новости', 'все', 'новости', 'закрытая', 'лента', 'москва', 'января', 'противники', 'передачи', 'исаакиевского', 'собора', 'рпц', 'намерены', 'выйти', 'на', 'митинг', 'января', 'с.-петербург', 'янв', 'риа', 'новости', 'активисты', 'выступающие', 'против', 'передачи', 'исаакиевского', 'собора', 'русской', 'православной', 'церкви', 'а', 'также', 'депутаты', 'законодательного', 'собрания', 'петербурга', 'в', 'пятницу', 'в', 'ходе', 'встречи', 'на', 'исаакиевской', 'площади', 'возле', 'собора', 'договорились', 'подать', 'заявку', 'на', 'проведение', 'митинга', 'января', 'сообщил', 'риа', 'новости', 'член', 'городского', 'парламента', 'борис', 'вишневский', 'заявка', 'на', 'проведение', 'митинга', 'на', 'марсовом', 'поле', 'января', 'уже', 'готова', 'мы', 'намерены', 'подать', 'ее', 'в', 'понедельник', 'сказал', 'он', 'во', 'вторник', 'пресс-секретарь', 'губернатора', 'санкт-петербурга', 'андрей', 'кибитов', 'сообщил', 'что', 'исаакиевский', 'собор', 'будет', 'передан', 'рпц', 'в', 

## Лемматизация с помощью Pymorphy2 ##

Для лемматизации нам понадобится еще один модуль, поэтому импортируем его. Затем пишем функцию, которая возвращает нам строку из лемм, создаем список лемм.

In [10]:
import pymorphy2

In [11]:
morph = pymorphy2.MorphAnalyzer() 
cache = dict() # ускоряем разбор слов, избавившись от повторной работы

def lemmatizer(token_list):
    lemmas = ''
    for word in token_list:
        if word in cache:
            lemma = cache[word]
        else:
            lemma = morph.parse(word)[0].normal_form
            cache[word] = lemma
        lemmas += lemma + ' '
    return lemmas
word_list = clean_news(news[2])
print(lemmatizer(word_list))

аргумент и факт aif.ru москва январь оппозиция провести митинг против передача исаакиевский собор рпц протестный мероприятие собрать около тысяча человек москва январь аиф-москва депутат законодательный собрание санкт-петербург от фракция яблоко борис вишневский сообщить что январь перед исаакиевский собор быть провести митинг протест против передача собор русский православный церковь передавать рбк сообщаться что мероприятие стать один в серия протестный акция участник мероприятие стать около тысяча человек кроме вишневский организатор выступить депутат заксобрание максим резник партия рост и алексей ковалев справедливый россия также депутат добавить что в настоящее время готовиться иск о оспаривание передача собор так как речь идти о грубый нарушение закон о передача религиозный организация имущество религиозный назначение находиться в государственный или муниципальный собственность ранее сообщаться что собор быть передать в безвозмездный пользование рпц по просьба патриарх кирилл дв

Поскольку CountVectorizer из модуля sklearn принимает на вход списки, мы создаем список лематизированных текстов. 

In [12]:
lemma_list = []

for item in news:
    tokens = clean_news(item)
   # token_list.append(tokens)
    lemmas = lemmatizer(tokens)
    lemma_list.append(lemmas)



## Извлечем все event_id  для того, чтобы соотнести их со списками лемм ##

In [13]:
event_ids = df_unique['event_id'].values
print(event_ids)

[ 1  1  1 ..., 28 28 28]


In [14]:
dataset = list(zip(event_ids, lemma_list))

In [15]:
df_lemma = pd.DataFrame(data = dataset, columns = ['event_id', 'lemmas'])
print(df_lemma[0:4])

   event_id                                             lemmas
0         1  в петербург пройти митинг против передача исаа...
1         1  lenta.co москва январь ситуация с передача иса...
2         1  аргумент и факт aif.ru москва январь оппозиция...
3         1  google новость топ москва январь акция против ...


# Шаг 2. Эксперименты#

## Кластеризация K-means ##

In [16]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import *
from sklearn.pipeline import *
from sklearn.preprocessing import Normalizer
from sklearn.metrics import *
from sklearn.cluster import *
import numpy as np

Выполним кластеризацию _k-means_ на 28 кластеров без использования текстовых преобразований нормалайзера и TF-IDF, а в последующих шагах посмотрим, как изменилась точность кластеризации после применения каждого метода. Для `min_df`, `max_df`, `ngram_range` и `analyzer` мы выбрали те параметры, при которых удалось получить наилучшие результаты.

## Запустим кластеризацию без дополнительных параметров ##

In [25]:
pipeline = Pipeline([
    ('vect', CountVectorizer(max_df = 0.9, min_df = 3, stop_words = stopwords_rus, analyzer = 'word')),
    #('tfidf', TfidfTransformer()),
    #('svd', TruncatedSVD(n_components = 1500)),
    #('norm', Normalizer() ),
    ('clust', KMeans(n_clusters = 28, random_state = 42))
])

# pipeline.fit(df_unique['text'].values)
pipeline.fit(df_lemma['lemmas'].values)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['и', 'в', ...=28, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0))])

Мы зафиксировали random state для того, чтобы точнее сравнивать точность методов между собой: 

**` KMeans(n_clusters = 28, random_state = 42)`**.

In [26]:
print(len(pipeline.named_steps['vect'].vocabulary_))

13601


In [27]:
clust_labels = pipeline.named_steps['clust'].labels_
labels = df_lemma['event_id']

print("Homogeneity:", homogeneity_score(labels, clust_labels))
print("Completeness:", completeness_score(labels, clust_labels))
print("V-measure",  v_measure_score(labels, clust_labels))
print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))

Homogeneity: 0.299238061546
Completeness: 0.59569106146
V-measure 0.398363253422
Adjusted Rand-Index: 0.0507037962299


##  Не очень =( ##

## Попробуем сингулярное разложение ##

In [18]:
pipeline = Pipeline([
    ('vect', CountVectorizer(max_df = 0.9, min_df = 3, stop_words = stopwords_rus, analyzer = 'word')),
    #('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(n_components = 1500)),
    #('norm', Normalizer() ),
    ('clust', KMeans(n_clusters = 28, random_state = 42))
])

pipeline.fit(df_lemma['lemmas'].values)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['и', 'в', ...=28, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0))])

In [19]:
clust_labels = pipeline.named_steps['clust'].labels_
labels = df_lemma['event_id']

print("Homogeneity:", homogeneity_score(labels, clust_labels))
print("Completeness:", completeness_score(labels, clust_labels))
print("V-measure",  v_measure_score(labels, clust_labels))
print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))

Homogeneity: 0.236509918002
Completeness: 0.572003668854
V-measure 0.334650012113
Adjusted Rand-Index: 0.023971397177


**Итак без использования TF-IDF и нормалайзера мы получили очень маленькую точность даже при использовании SVD.** Добавим TF-IDF преобразование.

In [20]:
pipeline = Pipeline([
    ('vect', CountVectorizer(max_df = 0.9, min_df = 3, stop_words = stopwords_rus, analyzer = 'word')),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(n_components = 1500)),
    #('norm', Normalizer() ),
    ('clust', KMeans(n_clusters = 28, random_state = 42))
])

pipeline.fit(df_lemma['lemmas'].values)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['и', 'в', ...=28, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0))])

In [21]:
clust_labels = pipeline.named_steps['clust'].labels_
labels = df_lemma['event_id']

print("Homogeneity:", homogeneity_score(labels, clust_labels))
print("Completeness:", completeness_score(labels, clust_labels))
print("V-measure",  v_measure_score(labels, clust_labels))
print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))

Homogeneity: 0.944437942137
Completeness: 0.936065114609
V-measure 0.940232888616
Adjusted Rand-Index: 0.852724264627


**С TF-IDF преобразованием мы получили уже гораздо большую точность -- 85%. Уже гораздо лучше** Теперь добавим нормализацию.

In [27]:
pipeline = Pipeline([
    ('vect', CountVectorizer(max_df = 0.9, min_df = 3, stop_words = stopwords_rus, analyzer = 'word')),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(n_components = 1500)),
    ('norm', Normalizer() ),
    ('clust', KMeans(n_clusters = 28, random_state = 42))
])

# pipeline.fit(df_unique['text'].values)
pipeline.fit(df_lemma['lemmas'].values)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['и', 'в', ...=28, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0))])

In [28]:
clust_labels = pipeline.named_steps['clust'].labels_
labels = df_lemma['event_id']

print("Homogeneity:", homogeneity_score(labels, clust_labels))
print("Completeness:", completeness_score(labels, clust_labels))
print("V-measure",  v_measure_score(labels, clust_labels))
print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))

Homogeneity: 0.944448544976
Completeness: 0.93995242123
V-measure 0.942195119295
Adjusted Rand-Index: 0.83850548439


**Получаем точность 83%. Не сильно изменилась с предыдущего шага, даже слегка уменьшилась** 

In [22]:
confusion_matrix(labels, clust_labels)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  0,   0,   0,   0, 100,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  43,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          8,   0,   0],
       [  0,   0,   0,   0,   0,   0,  84,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  62,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
    

Итак, максимальный Rand Index, который мы получили составил 85%. Чтобы оценить роль нормализации текста, попробуем проделать кластеризацию текста без предобработки.

In [33]:
pipeline = Pipeline([
    ('vect', CountVectorizer(max_df = 0.9, min_df = 0.1, ngram_range = [2, 3], analyzer = 'char_wb')),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(n_components = 50)),
    ('norm', Normalizer() ),
    ('clust', KMeans(n_clusters = 28))
])

pipeline.fit(df_unique['text'].values)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='char_wb', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=0.1,
        ngram_range=[2, 3], preprocessor=None, stop_words=None,
        ...8, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0))])

In [34]:
clust_labels = pipeline.named_steps['clust'].labels_
labels = df_lemma['event_id']

print("Homogeneity:", homogeneity_score(labels, clust_labels))
print("Completeness:", completeness_score(labels, clust_labels))
print("V-measure",  v_measure_score(labels, clust_labels))
print("Adjusted Rand-Index:",  adjusted_rand_score(labels, clust_labels))

Homogeneity: 0.898025624474
Completeness: 0.881114598483
V-measure 0.88948974041
Adjusted Rand-Index: 0.757615603852


**Как видим, предобработка текста добавила точности в 10%**

# Шаг 3. Общие направления новостей #

Заимпортируем файл с классификацией резонансных событий. Для того, чтобы в последующих шагах соотнести новые кластеры со старыми событиями.

In [23]:
events = pd.read_csv('events.csv', sep = ',', header = 0)
events.head()

,id,date,name
0,1,2017-01-10 00:00:00,Власти Петербурга согласились передать РПЦ Иса...
1,2,2017-01-20 00:00:00,Дональд Трамп вступил в должность президента США.
2,3,2017-02-20 00:00:00,Скоропостижно скончался постпред России при ОО...
3,4,2017-03-02 00:00:00,Вышел фильм Навального «он Вам не димон»
4,5,2017-03-14 00:00:00,CNN показала фильм «Владимир Путин — самый вли...


In [24]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(max_df = 0.9, min_df = 3, stop_words = stopwords_rus, analyzer = 'word')),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD(n_components = 1500)),
    ('norm', Normalizer() ),
    ('clust', KMeans(n_clusters = 7, random_state = 42))
])

pipeline2.fit(df_lemma['lemmas'].values)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=3,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=['и', 'в', ...s=7, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=42, tol=0.0001, verbose=0))])

In [25]:
clust_labels2 = pipeline2.named_steps['clust'].labels_
text_labels = events.name.loc[labels-1]
text_labels.groupby(clust_labels2).value_counts()

   name                                                                                                    
0  Акции протеста 12 июня                                                                                      100
   Власти Петербурга согласились передать РПЦ Исаакиевский собор.                                              100
   Митинг в москве против коррупции                                                                            100
   В центре Киева был убит бывший депутат Госдумы РФ от КПРФ Денис Вороненков                                   73
   Вышел фильм Навального «он Вам не димон»                                                                     62
   Митинг против Реновации в Москве                                                                              8
   Несанкционированные акции в Москве апрель                                                                     6
   Дональд Трамп вступил в должность президента США.                                   

## Интерпретация результатов ##

В результате кластеризации на 7 кластеров методом KMeans получены основные направления новостей:
    
* 0 - Акции протеста, политика
* 1 - Выборы, голосования
* 2 - Спорт, соревнования
* 3 - Смерть
* 4 - Законопроект
* 5 - Путин, президенты
* 6 - Теракты, ураганы
    
Есть единичные случаи попадания событий в несоответствующие кластеры, но, в основном, события, попавшие в один кластер, действительно объединены общей темой.